In [ ]:
# @-*- coding = utf-8 _*_
# @Time : 2021/4/28 20:35
# @Author : 杨昌军
# @File : textmining.ipynb
# @Software : PyCharm

In [75]:
import sys #导入sys模块，以实现在指定地方结束程序
import pandas as pd
import numpy as np
import jieba
import jieba.posseg as pseg
import jieba.analyse
import re
from collections import Counter  #导入collections模块中的counter动作以进行词频统计。

In [85]:
def excel_to_list(excel_file,col_name):  #定义一个函数，将excel中的某一列转化为list列表。
    comments_text_dataframe = pd.read_excel(excel_file, converters={
        'id': str,'text':str})  # 读取excel_file所指称的文件，并将id列内容转换为字符串格式（此前为数值格式）。此时comments_text_dataframe是个dataframe。
    comment_array = np.array(comments_text_dataframe[
                                 col_name])  # 这里调用一个numpy里的array函数将dataframe里指定列的内容转换为数组。一行一个数据(子元素)，dataframe指定列里有多少行，这个数组就包含多少个数据。
    comment_list = comment_array.tolist()  # 这一步是将用array函数调用的数组通过.tolist()方法转化为列表。此时comments_list为列表，列表中每个元素都是一条字符串形式的评论。
    return comment_list,comments_text_dataframe  #返回一个包含指定列数据的列表、一个包含导入的excel全部内容的dataframe、指定列的列名。
def filter_sentence(data,filter_word): #定义一个函数，筛选包含指定词语的句子。
    comment_filtered_list = []
    for screen in data:
        screen = str(screen)
        if screen.find(filter_word) != -1:
            comment_filtered_list.append(screen)
    return comment_filtered_list
def save_all_filtered_data_to_excel(data_df,col_name,filter_word,save_filename):  #定义一个函数，将筛选过的数据存入excel。
    if save_filename == '':
        save_filename = (input("请输入你存储筛选过的数据的Excel文件名，不用输入后缀名“.xlsx”：")).strip()
        print(f"你准备将筛选过的数据存入{save_filename}.xlsx文件中，请选择此文件的保存路径。")
        save_path = slc_pth(get_pf()) #选择保存路径。
    data_frame = data_df.loc[data_df[col_name].str.contains(filter_word, na=False)] #在text列中查找有“生活”字符串的数据。
    data_frame.to_excel(f'{save_path}\\{save_filename}.xlsx',index=False)
    print(f'\033[5;30;47m…………筛选后的数据已保存至\033[1;46m{save_path}\033[0m下的“{save_filename}.xlsx”中…………\033[0m')
def clean_text(comments_list,regular_expression): #定义一个函数，进行文本清理。
    text_str = ''.join(str(comments_list))  #通过.join方法合并。
    pattern = re.compile(f'{regular_expression}')
    text_str = re.sub(pattern,'',text_str)
    return text_str
def word_segment_and_pos_tagging(new_words, stop_words, text_str):  #定义一个函数，先加载自定义词典，再加载停用词词典，然后对输入的字符串text_str进行分词，最终输出为列表。
    jieba.enable_paddle()  #利用飞桨Paddle深度学习框架，训练序列标注（双向GRU）网络模型实现分词及词性标注。
    jieba.load_userdict(new_words)    #加载自定义词典。
    stopwords = [line.strip() for line in open(stop_words, encoding='UTF-8').readlines()]   #加载停用词词典，并创建一个停用词列表。此时，stopwords是一个列表。
    print("正在利用飞桨paddle模式对文本进行分词和词性标注，需要一定时间，请稍等。")
    seg_and_tagging_all_word_list = pseg.lcut(text_str,use_paddle=True) #分词并标注词性，默认开启飞桨paddle模式下的分词与词性标注功能，因为对比过开与不开的效果，感觉确实开后分词与标注更精确，具体如何还要研究。
    segged_and_tagged_word_pair_list = []  #既包含词又包含词性的列表。
    for tagging_word in seg_and_tagging_all_word_list:
        if tagging_word.word not in stopwords:  #去除停用词，tagging_word.word为取tagging_word这个pair元组里word的值。
            segged_and_tagged_word_pair_list.append(tagging_word)
    segged_word_list = []  #只包含词，不包含词性。
    for seged_and_tagged_word in segged_and_tagged_word_pair_list:
        seged_word = seged_and_tagged_word.word
        segged_word_list.append(seged_word)
    return  segged_and_tagged_word_pair_list,segged_word_list  #segged_and_tagged_word_pair_list是一个既包含词又包含词性的list，segged_word_list仅包含词，不包含词性。
def sort_word_frequency(data):  #此函数用于统计词频并进行排序。返回的是一个排好序的列表。
    word_frequency_dic = dict(Counter(data))  # 用Counter()统计列表word_list中的词频，并转化为字典word_frequency_dic。注意，这里的Counter的首字母必须大写，否则会出错。
    word_frequency_dic_sorted = sorted(word_frequency_dic.items(), key=lambda x: x[1],reverse=True)  # [:100] #这里是先用字典的.items()方法将字典转换为列表，然后用列表的sorted方法对列表中的元素进行排序，后面的都是参数。
    return word_frequency_dic_sorted
def show_the_highest_ranked_word(data): #此函数用于直接显示排序最高的单词，可以不用。
    num = int(input("你想显示前多少个词频最高的单词/词语："))
    for i in range(num):
        word, count = sort_word_frequency(data)[i]
        print("{0:<10}{1:>5}".format(word, count))  #{0:<10}{1:>5}指定变量值的输出格式，其含义为输出第一个（0）变量时，左对齐（<），最小宽度为10，第二个（1）右对齐（>），最小宽度为5。

In [86]:
excel_file = 'E:\\PycharmProjects\\Demo0410\\work_data_liziqi\\liziqi_total_comments.xlsx' #可以直接填写要处理的excel文件的绝对路径。
col_name = 'text'  #可以直接填写此excel文件中要处理的数据所在的列名。
filter_word = '生活'  #筛选词
news_words = 'E:\\PycharmProjects\\Demo0410\\work_data_liziqi\\new_words.txt'    #自定义词典。
stop_words = 'E:\\PycharmProjects\\Demo0410\\work_data_liziqi\\stop_words.txt'    #停用词词典。
stop_words_after_filtered = 'E:\\PycharmProjects\\Demo0410\\work_data_liziqi\\stop_words_after_filtered.txt'  #相当于上面的停用词词典，主要是加了“生活”一词。
re_expression = '\n|\[.{1,8}\]|[^\u4e00-\u9fa5]'

In [87]:
comment_list, data_df = excel_to_list(excel_file, col_name)  #调用一开始定义的excel_to_list函数。注意，此函数会返回两个值，一个是包含导入的excel指定列数据的列表，一个是包含导入的excel全部数据的dataframe。
comment_filtered_list = filter_sentence(comment_list,filter_word)

In [88]:
cleaned_txt = clean_text(comment_filtered_list, re_expression)
segged_and_tagged_word_pair_list,segged_word_list = word_segment_and_pos_tagging(news_words, stop_words_after_filtered, cleaned_txt)
#……………………准备工作，分词和词性标注完成…………………………

Paddle enabled successfully......


正在利用飞桨paddle模式对文本进行分词和词性标注，需要一定时间，请稍等。


In [112]:
jieba.load_userdict(news_words)  #加载新词词典。
jieba.analyse.set_stop_words(stop_words_after_filtered) #加载停用词典。
tf_idf_comment_original_text = ''.join(str(comment_filtered_list))  #将过滤后的语料连接成字符串。
tf_idf_re_expression = '\[.{1,8}\]'  #表示表情包的正则式。
tf_idf_comment_text = clean_text(tf_idf_comment_original_text,tf_idf_re_expression)  #去掉表情包里的字符。
jieba.analyse.set_idf_path('idf.txt.big')  #使用从jieba下载的自定义逆向文件频率（IDF）语料库，经验证，关键词提取明显更准确。
tf_idf_word = jieba.analyse.extract_tags(tf_idf_comment_text,topK=20,withWeight=True,allowPOS=('ns','nr', 'n', 'nt','nw','nz','vn', 'vd','v','a','ad','an','d','m','q')) #参数allowPOS=为仅包括指定词性的词，这里主要是包括各种名词、动词、形容词、副词、量词。
#进行主题抽取。

[('农村', 0.3155804820589552),
 ('田园生活', 0.20370803603805576),
 ('向往', 0.14137115069307696),
 ('美好', 0.1391448333593277),
 ('羡慕', 0.13747509535901578),
 ('喜欢', 0.11966455668902183),
 ('享受', 0.08960927268340704),
 ('真的', 0.08905269334996974),
 ('感觉', 0.0868263760162205),
 ('农活', 0.08404347934903393),
 ('过成', 0.08348690001559662),
 ('看到', 0.06790267867935192),
 ('城市', 0.05955398867779226),
 ('幸福', 0.05788425067748033),
 ('一种', 0.056771092010605706),
 ('辛苦', 0.05621451267716839),
 ('农民', 0.055657933343731085),
 ('唯美', 0.05510135401029377),
 ('真实', 0.05510135401029377),
 ('小时候', 0.053431616009981836)]

[('生活', 0.1115435661767247),
 ('农活', 0.07272484640124853),
 ('转发', 0.05644889640805884),
 ('花生', 0.05075694031921688),
 ('农村', 0.0484278077762081),
 ('喜欢', 0.04432309888878171),
 ('能干', 0.03901361175097409),
 ('感觉', 0.03832620489029002),
 ('团队', 0.03647975367538351),
 ('真的', 0.03643553385220768),
 ('厉害', 0.03499240676556057),
 ('更新', 0.03315248874063557),
 ('田园生活', 0.0305244110901532),
 ('美好', 0.029680196578581383),
 ('唯美', 0.02963212913680822),
 ('勤劳', 0.02945898617149569),
 ('羡慕', 0.02819266766813838),
 ('美食', 0.027634978346131975),
 ('享受', 0.026889109191525852),
 ('向往的', 0.026349842454714584)]

[('李子柒', 0.17767676966229307),
 ('捂脸', 0.1330587446933824),
 ('玫瑰', 0.12284036838166847),
 ('生活', 0.10290484326991631),
 ('子柒', 0.08311200729502966),
 ('农活', 0.06673072668981452),
 ('转发', 0.05179627135084824),
 ('花生', 0.046573457073602574),
 ('农村', 0.04443629605033497),
 ('喜欢', 0.04097341180518921),
 ('向往', 0.03861391096172162),
 ('能干', 0.03579803590884028),
 ('感觉', 0.03520481821393355),
 ('团队', 0.03464834004603039),
 ('爱慕', 0.03458664818132684),
 ('真的', 0.03346992840094211),
 ('厉害', 0.0321082662616851),
 ('更新', 0.030774233550347276),
 ('田园生活', 0.02800853126022673),
 ('美好', 0.027589565094159746)]